In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%pylab inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import tensorflow as tf

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
X_train = df.drop(labels = ["label"],axis = 1)
X_train

In [ ]:
Y_train = df["label"]

In [ ]:
np.shape(X_train)

In [ ]:
np.sqrt(784)

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)

In [ ]:
np.shape(X_train)

In [ ]:
X_train[0]

In [ ]:
plt.imshow(X_train[0][:,:,0])
Y_train[0]

In [ ]:
from keras.utils import to_categorical
Y_train = to_categorical(Y_train)

In [ ]:
Y_train[0]

In [ ]:
X_train = X_train / 255.0

In [ ]:
X_train[0]

In [ ]:
plt.imshow(X_train[0][:,:,0])
Y_train[0]

In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import BatchNormalization
from keras import optimizers
from keras.layers import Activation

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 64, kernel_size = (4,4),padding = 'Same', activation ='relu'))
model.add(Dropout(0.2))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization(axis = -1))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Dropout(0.2))
model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Dropout(0.2))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(BatchNormalization(axis = -1))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model

In [ ]:
for layer in model.layers:
    print(layer , ": " ,layer.output_shape)

In [ ]:
ADAMAX = optimizers.Adamax(lr = 0.002, beta_1 = 0.9, beta_2 = 0.999)

In [ ]:
model.compile(optimizer = ADAMAX , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 20
batch_size = 64
steps = np.shape(X_train)
steps = steps[0]/64

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=2)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10, 
        zoom_range = 0.1,  
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False)  


datagen.fit(X_train)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_train,Y_train))
dataset = dataset.cache()
dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
dataset = dataset.repeat()
dataset = dataset.batch(64, drop_remainder=True)


In [ ]:
epoch_info = model.fit(datagen.flow(X_train,Y_train, batch_size=64), steps_per_epoch=X_train.shape[0]/64 , epochs=100 , validation_data=(X_val, Y_val), verbose=1)

In [ ]:
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test = test/255.0

In [ ]:
test.head()

In [ ]:
test = test.values.reshape(-1,28,28,1)

In [ ]:
plt.imshow(test[0][:,:,0])

In [ ]:
result = model.predict(test)
result

In [ ]:
result = np.argmax(result,axis = 1)
result = pd.Series(result,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),result],axis = 1)
submission.to_csv("cnn_mnist_datagen8.csv",index=False)